In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import csv
data={}
test_data = {}
i=0
curr = ''
with open('quartileData\\taskData_bdiTop.csv', 'r' ) as theFile:
    reader = csv.reader(theFile)
    headers = next(reader, None)
    for line in reader:
        if line[70] != curr:
            if i > 0:
                block = [
                    {
                        'action': np.array([actions])-1,
                        'state': np.array([states]),
#                         'reward': np.array([rewards]),
                        'id': curr,

                        'block': 0
                    }

                ]
#                 if i<226:
                data[curr] = block
#                 else:
                test_data[curr] = block
            curr = line[70]
            i += 1
            actions = []
            states = []
            rewards = []
        actions.append(int(float(line[8])))
        states.append([int(float(line[3])),int(float(line[4]))])
        rewards.append(int(float(line[10])))

In [3]:
data

{'A002160837SWJFPIAI7L7': [{'action': array([[0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
           1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
           1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
           1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
           1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
           1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
           0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
           0, 1, 0, 1, 1, 1]]),
   'state': array([[[ 7,  2],
           [ 2,  7],
           [ 7,  1],
           [ 1,  7],
           [ 1,  7],
           [ 1,  7],
           [ 2,  7],
           [ 1,  7],
           [ 7,  1],
           [ 7,  2],
           [ 2,  1],
           [ 2,  7],
           [ 1,  7],
           [ 2,  4],
           [ 4,  2],
           [ 4,  2],
           [ 4,  2],
           [ 4,  2],
       

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn_noReward.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn_noReward.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/taskData_bdiTop_noReward/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-08-12 14:13:24,724 - DeepRL - DEBUG - version control: (None, None)
2021-08-12 14:13:24,724 - DeepRL - DEBUG - learning rate: 0.01
2021-08-12 14:13:24,724 - DeepRL - DEBUG - global iters: 50
2021-08-12 14:13:24,724 - DeepRL - DEBUG - training data-points: 250
2021-08-12 14:13:24,729 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-08-12 14:13:25,169 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/taskData_bdiTop_noReward/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 14:13:40,839 - DeepRL - DEBUG - global iter =    0 total obj: 27444.9614
2021-08-12 14:13:57,184 - DeepRL - DEBUG - global iter =    1 total obj: 27391.4838
2021-08-12 14:14:11,908 - DeepRL - DEBUG - global iter =    2 total obj: 27273.3812
2021-08-12 14:14:26,515 - DeepRL - DEBUG - global iter =    3 total obj: 27007.7210
2021-08-12 14:14:41,137 - DeepRL - DEBUG - global iter =    4 total obj: 26671.4963
2021-08-12 14:14:55,751 - DeepRL - DEBUG - global iter =    5 total obj: 26431.3880
2021-08-12 14:15:10,402 - DeepRL - DEBUG - global iter =    6 total obj: 26281.6229
2021-08-12 14:15:25,212 - DeepRL - DEBUG - global iter =    7 total obj: 26140.8049
2021-08-12 14:15:39,870 - DeepRL - DEBUG - global iter =    8 total obj: 26062.8559
2021-08-12 14:15:54,517 - DeepRL - DEBUG - global iter =    9 total obj: 25995.1531
2021-08-12 14:16:09,275 - DeepRL - DEBUG - global iter =   10 total obj: 25902.4218
2021-08-12 14:16:23,960 - DeepRL - DEBUG - global iter =   11 total obj: 258

INFO:tensorflow:../results/taskData_bdiTop_noReward/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
# from actionflow.data.data_process import DataProcess
# train_merged = DataProcess.merge_data(data)
# train_merged

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-08-12 14:37:26,244 - DeepRL - DEBUG - version control: (None, None)
2021-08-12 14:37:26,246 - DeepRL - DEBUG - learning rate: 0.01
2021-08-12 14:37:26,247 - DeepRL - DEBUG - global iters: 50
2021-08-12 14:37:26,248 - DeepRL - DEBUG - training data-points: 250
2021-08-12 14:37:26,249 - DeepRL - DEBUG - test data-points: 250
2021-08-12 14:37:42,790 - DeepRL - DEBUG - opt started...
2021-08-12 14:37:42,792 - DeepRL - DEBUG - started testing...
2021-08-12 14:46:42,817 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_bdiTop_noReward/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 14:47:27,176 - DeepRL - DEBUG - global iter =    0 total obj: 27431.2372
2021-08-12 14:47:46,657 - DeepRL - DEBUG - global iter =    1 total obj: 27411.6820
2021-08-12 14:48:05,209 - DeepRL - DEBUG - global iter =    2 total obj: 27166.4933
2021-08-12 14:48:25,863 - DeepRL - DEBUG - global iter =    3 total obj: 26804.2244
2021-08-12 14:48:46,518 - DeepRL - DEBUG - global iter =    4 total obj: 26599.7561
2021-08-12 14:49:07,382 - DeepRL - DEBUG - global iter =    5 total obj: 26505.8966
2021-08-12 14:49:28,224 - DeepRL - DEBUG - global iter =    6 total obj: 26402.9482
2021-08-12 14:49:49,347 - DeepRL - DEBUG - global iter =    7 total obj: 26316.1046
2021-08-12 14:50:10,139 - DeepRL - DEBUG - global iter =    8 total obj: 26234.0537
2021-08-12 14:50:30,861 - DeepRL - DEBUG - global iter =    9 total obj: 26173.1542
2021-08-12 14:50:30,863 - DeepRL - DEBUG - started testing...
2021-08-12 14:54:59,700 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_bdiTop_noReward/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 14:55:22,220 - DeepRL - DEBUG - global iter =   10 total obj: 26115.2054
2021-08-12 14:55:42,610 - DeepRL - DEBUG - global iter =   11 total obj: 26070.1294
2021-08-12 14:56:03,193 - DeepRL - DEBUG - global iter =   12 total obj: 26040.2485
2021-08-12 14:56:23,898 - DeepRL - DEBUG - global iter =   13 total obj: 25977.9715
2021-08-12 14:56:44,551 - DeepRL - DEBUG - global iter =   14 total obj: 25925.4898
2021-08-12 14:57:05,780 - DeepRL - DEBUG - global iter =   15 total obj: 25887.3520
2021-08-12 14:57:26,577 - DeepRL - DEBUG - global iter =   16 total obj: 25875.2703
2021-08-12 14:57:47,817 - DeepRL - DEBUG - global iter =   17 total obj: 25839.6843
2021-08-12 14:58:14,425 - DeepRL - DEBUG - global iter =   18 total obj: 25783.7912
2021-08-12 14:58:40,829 - DeepRL - DEBUG - global iter =   19 total obj: 25755.9039
2021-08-12 14:58:40,831 - DeepRL - DEBUG - started testing...
2021-08-12 15:03:00,794 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/taskData_bdiTop_noReward/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 15:11:20,057 - DeepRL - DEBUG - global iter =   30 total obj: 25635.8506
2021-08-12 15:11:40,026 - DeepRL - DEBUG - global iter =   31 total obj: 25541.6842
2021-08-12 15:11:59,458 - DeepRL - DEBUG - global iter =   32 total obj: 25529.1428
2021-08-12 15:12:20,736 - DeepRL - DEBUG - global iter =   33 total obj: 25505.3404
2021-08-12 15:12:41,589 - DeepRL - DEBUG - global iter =   34 total obj: 25449.9053
2021-08-12 15:13:02,352 - DeepRL - DEBUG - global iter =   35 total obj: 25423.0200
2021-08-12 15:13:23,299 - DeepRL - DEBUG - global iter =   36 total obj: 25434.2693
2021-08-12 15:13:44,129 - DeepRL - DEBUG - global iter =   37 total obj: 25382.4443
2021-08-12 15:14:05,022 - DeepRL - DEBUG - global iter =   38 total obj: 25395.2933
2021-08-12 15:14:25,891 - DeepRL - DEBUG - global iter =   39 total obj: 25428.7457
2021-08-12 15:14:25,893 - DeepRL - DEBUG - started testing...
2021-08-12 15:18:58,126 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_bdiTop_noReward/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 15:19:27,318 - DeepRL - DEBUG - global iter =   40 total obj: 25403.8205
2021-08-12 15:19:48,170 - DeepRL - DEBUG - global iter =   41 total obj: 25392.4755
2021-08-12 15:20:08,761 - DeepRL - DEBUG - global iter =   42 total obj: 25448.6429
2021-08-12 15:20:29,258 - DeepRL - DEBUG - global iter =   43 total obj: 25400.9857
2021-08-12 15:20:49,864 - DeepRL - DEBUG - global iter =   44 total obj: 25414.3924
2021-08-12 15:21:09,980 - DeepRL - DEBUG - global iter =   45 total obj: 25393.9349
2021-08-12 15:21:30,622 - DeepRL - DEBUG - global iter =   46 total obj: 25351.2285
2021-08-12 15:21:51,112 - DeepRL - DEBUG - global iter =   47 total obj: 25363.3250
2021-08-12 15:22:11,776 - DeepRL - DEBUG - global iter =   48 total obj: 25405.1221
2021-08-12 15:22:32,319 - DeepRL - DEBUG - global iter =   49 total obj: 25369.7150
2021-08-12 15:22:32,321 - DeepRL - DEBUG - opt finished.
2021-08-12 15:22:32,323 - DeepRL - DEBUG - started testing...
2021-08-12 15:27:42,766 - DeepRL - DEBUG 

INFO:tensorflow:../results/taskData_bdiTop_noReward/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
# test_data["S248"]